# Lung Cancer Prediction

We are using a dataset of three different kinds of images which the system will learn from 

In [1]:
from torch.utils.data import Dataset, DataLoader
import torch
from torchvision import transforms
import cv2
from sklearn.model_selection import train_test_split
from torch.nn import BCELoss
from tqdm import tqdm

C:\Users\ihsan\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import numpy as np
import glob
import matplotlib as plt
from PIL import Image
from torch.utils.tensorboard import SummaryWriter

In [3]:
!nvidia-smi

Thu Dec 22 00:50:45 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 527.56       Driver Version: 527.56       CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   45C    P8    N/A /  N/A |      0MiB /  4096MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
class LData(Dataset):
    def __init__(self, path, trans = None):
        self.location = path
        self.lgdata = [] #save data here
        self.labels = []# save each corresponding labels here
        self.lb = []
        self.trans = trans
        
    def __getitem__(self, serial):
            i=0
            for x in glob.glob(self.location+"\*"):
                self.labels.append(x)
                for y in glob.glob(self.labels[i]+"\*"):
                    self.lgdata.append(y)
                    self.lb.append(i)
                i+=1
            img = Image.open(self.lgdata[serial]).convert('L') # convert('RGB') to colored
            if self.trans is not None:
                img = self.trans(img)   
            return img, self.lb[serial]
    def __len__(self):
        return len(self.lb)
        

In [5]:
def BCE(labels, outputs):
    loss = torch.nn.CrossEntropyLoss()
#     outputs = outputs.squeeze(1)
    loss = loss(outputs, labels)
    return loss

def KLdiv(labels, outputs):
    loss = KLdiv()
    loss = loss(outputs, labels)
    return loss


In [6]:
def evaluate(model, valset):
    correct = 0
    total = 0
    model.eval()
    print("-"*44, "Evaluation", "-"*44)

    with tqdm(total=len(valset), colour = 'green') as t:
        for _, (inputs, labels) in enumerate(valset):
            if torch.cuda.is_available():
                inputs = inputs.to('cuda')
                labels = labels.to('cuda')
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            t.update()
        ta = 100 * correct/total
        tl= (100-ta)/100
        return ta, tl


In [7]:
import time

def train(model, trainset, valset, optimizer, criterion, epochs):
    best_accuracy = 0
    correct = 0
    total =0
    y = 0
    total_starttime = time.time()
    log_dir = '.' + '/tensorboard/'
    writer = SummaryWriter(log_dir=log_dir)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[3, 20, 30], gamma=0.2)
    for i in range(0, epochs):
        loss = 0
        print('-'*100)
        print("Epochs ",i, " out of ", epochs)
        print('-'*100)
        epochstart = time.time()
        scheduler.step(i)
        model.train()
        print("-"*45, "Training", "-"*45)
        with tqdm(total=len(trainset), colour = 'red') as t:
            for _, (inputs, labels) in enumerate(trainset):
                if torch.cuda.is_available():
                    inputs = inputs.to('cuda')
                    labels = labels.to('cuda')
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(labels, outputs)
                loss.backward()
                optimizer.step()                
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()
                
                t.set_postfix(loss='{:05.3f}'.format(loss), lr='{:05.6f}'.format(optimizer.param_groups[0]['lr']))
                t.update()
            epochend = time.time()        
        #----------------------Check Again---------------------------#
        train_accuracy = 100 * correct/total
        train_loss= (100-train_accuracy)/100
        #----------------------Check Again---------------------------#
        model.eval()
        del inputs
        del labels
        del outputs
        test_accuracy, test_loss = evaluate(model, valset)
        print("Train Accuracy: ", train_accuracy, "Train Loss: ", train_loss)
        print("Test Accuracy: ", test_accuracy, "Test Loss: ", test_loss)
        if test_accuracy >= best_accuracy:
            print('*'*10, "Congrats!!!! Higher Performance Achieved", '*'*10)
            torch.save({
            'state_dict': model.state_dict(),
            'optimizer' : optimizer.state_dict(),
            }, 'mbv2.pth.tar')
            best_accuracy = test_accuracy
        
        writer.add_scalar('Loss/train', train_loss,  i)
        writer.add_scalar('Loss/test', test_loss, i)
        writer.add_scalar('Accuracy/train', train_accuracy, i)
        writer.add_scalar('Accuracy/test', test_accuracy, i)

            
    total_endtime = time.time()
    total_endtime = total_endtime - total_starttime
    tm = time.gmtime(total_endtime)
    print("Time taken to complete training: ", tm.tm_mon," months", tm.tm_mday, " days", 
                    tm.tm_hour, " hours", tm.tm_min, " minutes", tm.tm_sec, "seconds")

In [8]:
import model.alexnet as alexnet
import model.resnet as resnet
import model.mobilenetv2 as mobilenet
import model.shufflenetv2 as shufflenet

def main():
    
    model = "ResNet18"
    criterion = "CE"
    dataset = "Base"
    if model == "AlexNet":
        model = alexnet.AlexNet(num_classes = 3).to('cuda')
        batch_size = 64
    elif model == "ResNet18":
        model = resnet.ResNet18(num_classes = 3).to('cuda')
        batch_size = 5
    elif model == "ShuffleNetV2":
        model = shufflnet.shufflenetv2(class_num = 3).to('cuda')
        batch_size = 10
    elif model == "ResNet50":
        model = resnet.ResNet50(num_classes = 3).to('cuda')
        batch_size = 2
    elif model == "ResNet152":
        model = resnet.ResNet152(num_classes = 3).to('cuda')
        batch_size = 5
    elif model == "MobileNetV2":
        model = mobilenet.mobilenetv2(class_num = 3).to('cuda')
        batch_size = 5
    
    if criterion == "CE":
        criterion = BCE
    elif criterion == "KLD":
        criterion = KLD
    else:
        criterion = ''
    epochs = 50
    optimizer = torch.optim.Adam(params = model.parameters(), lr = 0.001, capturable = True)#backpropagating
    if dataset == "Base": 
        path=".\L_dataset"
        transform = transforms.Compose([
                transforms.Resize((224,224)),
                transforms.ToTensor(),
            #         transforms.Normalize(0.2859, 0.3530)
        ])
        LD = LData(path, transform)
        LD[0]
        print("Total Images in the Dataset: ", len(LD))
        complete_set = torch.utils.data.DataLoader(LD, batch_size=1096, num_workers=0, shuffle=True)
        train_idx = valid_idx = 0
        image,labels = next(iter(complete_set))
        labels = labels.to('cpu').numpy()
        print("Labels Visual: ", labels)
        #stratified train-test-split to get same percentage from all the classes
        train_idx, valid_idx = train_test_split(np.arange(len(labels)), test_size=0.2,           #scikit learn train test split
                                                random_state=42, shuffle=True, stratify=labels)  #for class imbalance issue
        print("Train Set Size: ", len(train_idx))
        print("Validation Set Size: ", len(valid_idx))
        train_sampler = torch.utils.data.SubsetRandomSampler(train_idx)
        valid_sampler = torch.utils.data.SubsetRandomSampler(valid_idx)
        train_loader = torch.utils.data.DataLoader(LD, batch_size=batch_size, sampler=train_sampler)
        valid_loader = torch.utils.data.DataLoader(LD, batch_size=batch_size, sampler=valid_sampler)
        #training loop
        train(model, train_loader, valid_loader, optimizer, criterion, epochs)
        


In [ ]:
if __name__ == '__main__':
    main()

Total Images in the Dataset:  1097


C:\Users\ihsan\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\optim\lr_scheduler.py:131: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
C:\Users\ihsan\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\optim\lr_scheduler.py:156: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an

Labels Visual:  [2 2 1 ... 2 1 1]
Train Set Size:  876
Validation Set Size:  220
----------------------------------------------------------------------------------------------------
Epochs  0  out of  50
----------------------------------------------------------------------------------------------------
--------------------------------------------- Training ---------------------------------------------


100%|███████████████████████████████████████████████████████| 176/176 [01:53<00:00,  1.54it/s, loss=0.010, lr=0.001000]


-------------------------------------------- Evaluation --------------------------------------------


...

100%|███████████████████████████████████████████████████████| 176/176 [01:45<00:00,  1.67it/s, loss=0.000, lr=0.000200]


-------------------------------------------- Evaluation --------------------------------------------


100%|██████████████████████████████████████████████████████████████████████████████████| 44/44 [00:11<00:00,  3.89it/s]


Train Accuracy:  90.61390157280569 Train Loss:  0.09386098427194313
Test Accuracy:  96.81818181818181 Test Loss:  0.03181818181818187
----------------------------------------------------------------------------------------------------
Epochs  18  out of  50
----------------------------------------------------------------------------------------------------
--------------------------------------------- Training ---------------------------------------------


 68%|█████████████████████████████████████▌                 | 120/176 [01:11<00:33,  1.68it/s, loss=0.004, lr=0.000200]

In [ ]:
torch.cuda.empty_cache()

In [ ]:
!nvidia-smi